<a href="https://colab.research.google.com/github/matheusgondo/jiraanalytics/blob/main/JiraAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment setup and file download - Google Drive

Load libraries

Jira: Library used to access Jira

Panda: Library used for data manipulation and analysis.



In [ ]:
!pip install jira
!pip install -U -q PyDrive

#Import the libraries that will be used

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from collections import Counter
from typing import cast

from jira import JIRA
from jira.client import ResultList
from jira.resources import Issue

#set gdrive access to save data
from google.colab import drive

import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns
# import math
import datetime 
import json

pd.options.display.float_format = '{}'.format



In [ ]:
#jira methods
def init_jira(jiraServer,email,token):
  jira = JIRA(server=jiraServer, basic_auth=(email,token))
  return jira

def search_issues(jql,expand_changelog):
  issues = []
  n = 0
  if(expand_changelog):
    while True:
      tempissues = ResultList
      tempissues = jira.search_issues(jql,expand="changelog", json_result=True,maxResults=100,startAt=n)
      n+=100
      if not tempissues["issues"]:
        break
      else:
        issues.extend(tempissues["issues"])
  else:
    while True:
      tempissues = ResultList
      tempissues = jira.search_issues(jql,json_result=True,maxResults=100,startAt=n)
      n+=100
      if not tempissues["issues"]:
        break
      else:
        issues.extend(tempissues["issues"])
  return issues

def get_comments(id):
  commments = ResultList
  commments = jira.comments(id)
  return commments

def get_statuses():
  statuses = ResultList
  statuses = jira.statuses()
  return statuses

#fill dataframes
def issues_df(issues):
  keys = []
  summaries = []
  statuses = []
  timesspent = []
  issuetypes = []
  createddates = []
  sprints = []
  for issue in issues:
    sprintnames = ""
    keys.append(issue["key"])
    summaries.append(issue["fields"]["summary"])
    statuses.append(issue["fields"]["status"]["name"])
    timesspent.append(datetime.timedelta(seconds=issue["fields"]["aggregatetimespent"]+issue["fields"]["timespent"] if issue["fields"]["aggregatetimespent"] and issue["fields"]["timespent"] else 0))
    issuetypes.append(issue["fields"]["issuetype"]["name"])
    createddates.append(issue["fields"]["created"])
  df = pd.DataFrame({"key":keys,"summary":summaries,"status":statuses,"timespent":timesspent,"issuetype":issuetypes, "created":createddates})
  df["timespent"].fillna(pd.Timedelta(seconds=0))
  return df

def comments_df(keys):  
  comment_keys = []
  authors = []
  createddates = []
  bodies = []
  for key in keys:
    for comment in get_comments(key):
      comment_keys.append(key)
      authors.append(comment.author)
      createddates.append(comment.created)
      bodies.append(comment.body)
  df = pd.DataFrame({"key":comment_keys,"author":authors,"created date":createddates,"body":bodies})
  return df
  
def changelog_df(issues):  
  keys = []
  authorsnames = []
  authorsids = []
  dates = []
  fields = []
  fieldstype = []
  fieldsId = []
  froms = []
  fromStrings  = []
  tos = []
  toStrings = []
  for issue in issues:
      if issue["changelog"]["total"] >0:
        for history in issue["changelog"]["histories"] :
          for item in history["items"] :
            keys.append(issue["key"])
            authorsnames.append(history["author"]["displayName"])
            authorsids.append(history["author"]["accountId"])
            dates.append(history["created"])
            fields.append(item["field"])
            fieldstype.append(item["fieldtype"])
            if "fieldId" in item:
              fieldsId.append(item["fieldId"])
            else:
              fieldsId.append("")
            froms.append(item["from"])
            fromStrings.append(item["fromString"])
            tos.append(item["to"])
            toStrings.append(item["toString"])
  df = pd.DataFrame({"key":keys,"displayName":authorsnames,"accountId":authorsids,"created":dates,"field":fields,"fieldtype":fieldstype,"fieldId":fieldsId,"from":froms,"fromString":fromStrings,"to":tos,"toString":toStrings})
  return df

  

def transitions_df(changelogdf,issuesdf):  
  create_transitions = issuesdf.copy()
  create_transitions['toString'] = 'Product Backlog/To Do'
  transitions = changelogdf.loc[changelogdf["field"] == "status"]
  transitions = pd.concat([transitions, create_transitions])
  transitions = transitions.sort_values('created').drop_duplicates(subset=['key', 'toString'], keep='last')
  statuses = transitions.sort_values("created").pivot(index='key',values='created',columns='toString')
  statuses = statuses.astype('datetime64[ns]')
  statuses = statuses[transitions.sort_values('created').drop_duplicates(subset=['toString'], keep='first')["toString"]]
  return statuses

In [ ]:
jiraServer = ""
email = ""
token = ""
jql = ""

#init
jira = init_jira(jiraServer,email,token)

issues = json.loads(json.dumps(search_issues(jql,True)))

keys = []
for issue in issues["issues"]:
  keys.append(issue["key"])

#simplified issues view
issuesdf = issues_df(issues)

#retrieve comments
commentsdf = comments_df(keys)

#retrieve changelog expanded
changelogdf = changelog_df(issues)
  
#leadtime
transitionsdf = transitions_df(changelogdf,issuesdf)


In [ ]:
#Allows a more interactive dynamic filter
%reload_ext google.colab.data_table
#issuesdf#.groupby(['issuetype'])['timespent'].sum() 
#commentsdf
#calc lead time (interval between two statuses) 
#transitionsdf['in development [s]'] = (transitionsdf['In Development'] - transitionsdf['Developed']).apply(lambda x: x.seconds)

#save CSV in gdrive
transitionsdf.to_csv(r'/content/gdrive/MyDrive/jiraAnalytics/export_dataframe.csv', index = False, header=True)




In [ ]:
timespentdf =  issuesdf.groupby(['issuetype'])['timespent'].sum().reset_index(name ='Total timespent')
timespentdf["Total timespent"] = pd.to_timedelta(timespentdf["Total timespent"])
timespentdf['Total timespent'] = pd.to_numeric(timespentdf['Total timespent'].dt.total_seconds(), downcast='integer')

ax = timespentdf.plot.bar(x='issuetype', y='Total timespent', rot=0,figsize=(17,12))
